In [1]:
import libcst as cst
import intake
ds = intake.open_esm_datastore("/Users/u1166368/catalog/access-nri-intake-catalog/tests/data/esm_datastore/cmip-forcing-qv56.json")

/Users/u1166368/miniforge3/envs/access-nri-intake-test/lib/python3.13/site-packages/fastprogress/fastprogress.py:107: UserWarning: Couldn't import ipywidgets properly, progress bar will use console behavior
  warn("Couldn't import ipywidgets properly, progress bar will use console behavior")


In [2]:
ds.search(source_id='.*').to_dataset_dict()


--> The keys in the returned dictionary of datasets are constructed as follows:
	'file_type.project_id.institue_id.source_id.realm.frequency.table_id.mip_era.target_mip.variable_id.version'


ESMDataSourceError: Failed to load dataset with key='f.input4MIPs.na.MRI-JRA55-do-1-3.atmos.3hr.gn.CMIP6.OMIP.rlds.v20180412'
                 You can use `cat['f.input4MIPs.na.MRI-JRA55-do-1-3.atmos.3hr.gn.CMIP6.OMIP.rlds.v20180412'].df` to inspect the assets/files for this key.
                 

In [4]:
src_code = _i

In [5]:
src_tree = cst.parse_module(src_code)
print(src_code)

src_tree = cst.parse_module(src_code)
print(src_code)


In [6]:
from libcst.display import dump
print(dump(src_tree))

Module(
  body=[
    SimpleStatementLine(
      body=[
        Assign(
          targets=[
            AssignTarget(
              target=Name(
                value='src_tree',
              ),
            ),
          ],
          value=Call(
            func=Attribute(
              value=Name(
                value='cst',
              ),
              attr=Name(
                value='parse_module',
              ),
            ),
            args=[
              Arg(
                value=Name(
                  value='src_code',
                ),
              ),
            ],
          ),
        ),
      ],
    ),
    SimpleStatementLine(
      body=[
        Expr(
          value=Call(
            func=Name(
              value='print',
            ),
            args=[
              Arg(
                value=Name(
                  value='src_code',
                ),
              ),
            ],
          ),
        ),
      ],
    ),
  ],
)


In [9]:
class SearchCollector(cst.CSTVisitor):
    """
    A cst visitory that prints the search every time find one
    """
    def visit_Call(self, node: cst.Call) -> bool:
        if isinstance(node.func, cst.Call):
            print(f"Found search call at line {node.lineno}")

visitor = SearchCollector()
_ = src_tree.visit(visitor)

In [10]:
class ChainTraverser(cst.CSTVisitor):
    """
    Traverse method chains in evaluation order (inside-out)
    """
    METADATA_DEPENDENCIES = (cst.metadata.ParentNodeProvider,)
    
    def visit_Call(self, node: cst.Call) -> None:
        # Only process the outermost call to avoid duplicate processing
        if not self._is_nested_in_call(node):
            self._traverse_chain(node, [])
    
    def _is_nested_in_call(self, node: cst.Call) -> bool:
        """Check if this call is nested inside another call using parent metadata"""
        parent = self.get_metadata(cst.metadata.ParentNodeProvider, node)
        while parent:
            if isinstance(parent, cst.Call):
                return True
            parent = self.get_metadata(cst.metadata.ParentNodeProvider, parent, None)
        return False
    
    def _traverse_chain(self, node: cst.Call, chain: list) -> None:
        """Recursively traverse the chain from inside-out"""
        if isinstance(node.func, cst.Attribute):
            # This is a method call
            method_name = node.func.attr.value
            
            if isinstance(node.func.value, cst.Call):
                # There's a nested call - traverse it first
                self._traverse_chain(node.func.value, chain)
            elif isinstance(node.func.value, cst.Name):
                # Base object
                base_name = node.func.value.value
                print(f"1. Start with: {base_name}")
                chain.append(base_name)
            
            # Now process this method call
            print(f"{len(chain) + 1}. Call .{method_name}() -> returns some_type")
            chain.append(f".{method_name}()")

# You need to use MetadataWrapper to enable parent tracking
wrapper = cst.metadata.MetadataWrapper(src_tree)
visitor = ChainTraverser()
wrapper.visit(visitor)

1. Start with: cst
2. Call .parse_module() -> returns some_type


Module(
    body=[
        SimpleStatementLine(
            body=[
                Assign(
                    targets=[
                        AssignTarget(
                            target=Name(
                                value='src_tree',
                                lpar=[],
                                rpar=[],
                            ),
                            whitespace_before_equal=SimpleWhitespace(
                                value=' ',
                            ),
                            whitespace_after_equal=SimpleWhitespace(
                                value=' ',
                            ),
                        ),
                    ],
                    value=Call(
                        func=Attribute(
                            value=Name(
                                value='cst',
                                lpar=[],
                                rpar=[],
                            ),
                            at

In [11]:
ds.search(source_id='any').search(source_id='.*').to_dataset_dict()

/Users/u1166368/miniforge3/envs/access-nri-intake-test/lib/python3.13/site-packages/pydantic/_internal/_validate_call.py:111: UserWarning: There are no datasets to load! Returning an empty dictionary.
  res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))


{}

In [12]:
src_code_2 = _i

In [13]:
src_code_2
src_tree_2 = cst.parse_module(src_code_2)
print(src_code_2)

ds.search(source_id='any').search(source_id='.*').to_dataset_dict()


In [14]:
wrapper = cst.metadata.MetadataWrapper(src_tree_2)
visitor = ChainTraverser()
wrapper.visit(visitor)

1. Start with: ds
2. Call .search() -> returns some_type
3. Call .search() -> returns some_type
4. Call .to_dataset_dict() -> returns some_type


Module(
    body=[
        SimpleStatementLine(
            body=[
                Expr(
                    value=Call(
                        func=Attribute(
                            value=Call(
                                func=Attribute(
                                    value=Call(
                                        func=Attribute(
                                            value=Name(
                                                value='ds',
                                                lpar=[],
                                                rpar=[],
                                            ),
                                            attr=Name(
                                                value='search',
                                                lpar=[],
                                                rpar=[],
                                            ),
                                            dot=Dot(
                                     

In [15]:
import libcst as cst
import libcst.matchers as m

class SpecificChainMatcher(cst.CSTVisitor):
    def visit_Call(self, node: cst.Call) -> None:
        # Match specifically: ds.search(...).to_dataset_dict()
        search_to_dataset = m.Call(
            func=m.Attribute(
                attr=m.Name("to_dataset_dict"),
                value=m.Call(
                    func=m.Attribute(
                        attr=m.Name("search"),
                        value=m.Name("ds")
                    )
                )
            )
        )
        
        if m.matches(node, search_to_dataset):
            print("Found the exact pattern: ds.search(...).to_dataset_dict()")

# Test it
visitor = SpecificChainMatcher()
wrapper = cst.metadata.MetadataWrapper(src_tree)
_ = wrapper.visit(visitor)

src_code

'src_tree = cst.parse_module(src_code)\nprint(src_code)'

In [16]:
visitor = SpecificChainMatcher()
wrapper = cst.metadata.MetadataWrapper(src_tree_2)
_ = wrapper.visit(visitor)

src_code_2

"ds.search(source_id='any').search(source_id='.*').to_dataset_dict()"

In [30]:
import libcst as cst
import libcst.matchers as m

def extract_call_args(call_node : cst.Call) -> tuple[list, dict]:
    """
    Take a cst Call Node and extract the args and kwargs, into a tuple of (args, kwargs)
    """
    args = []
    kwargs = {}
    for arg in call_node.args:
        if arg.keyword is None:
            arg.append(arg.value)
        else:
            key = arg.keyword.value 
            kwargs[key] = arg.value.value
            # ^ Arg.value.value looks stupid, but arg.value is a cst Node itself
            # For catalogs, it's usually a cst.SimpleString or something like that,
            # so we could probably literal eval it?

    return args, kwargs

def format_args(args: list, kwargs: dict) -> str:
    """
    Format args and kwargs into a string representation
    """
    if args:
        args_str = ", ".join([arg for arg in args])
    if kwargs:
        kwargs_str = ", ".join([f"{k}={v}" for k, v in kwargs.items()])

    if not args:
        return kwargs_str if kwargs_str else ""
    return f"{args_str}, {kwargs_str}" if kwargs_str else args_str



class ChainSimplifier(cst.CSTTransformer):
    """
    Transform chained calls by removing intermediate method calls
    Example: ds.search(...).search(...).to_dataset_dict() 
    becomes: ds.to_dataset_dict()
    """
    
    def leave_Call(self, original_node: cst.Call, updated_node: cst.Call) -> cst.Call:
        # Use matcher to identify the pattern: any_method(search_call(...))
        search_pattern = m.Call(
            func=m.Attribute(
                value=m.Call(
                    func=m.Attribute(attr=m.Name("search"))
                )
            )
        )
        
        if m.matches(updated_node, search_pattern):
            # Extract the method name and inner call
            method_name = updated_node.func.attr.value
            inner_call = updated_node.func.value

            args, kwargs = extract_call_args(inner_call)
            
            print(f"Found chain: {inner_call.func.value.value}.{inner_call.func.attr.value}({format_args(args, kwargs)}) -> .{method_name}()")
            
            print(f"Removing .search({format_args(args, kwargs)}) from chain, keeping .{method_name}()")
            
            # Replace the value with the inner call's value
            # This effectively removes the search() call
            new_func = updated_node.func.with_changes(
                value=inner_call.func.value
            )
            return updated_node.with_changes(func=new_func)
        
        return updated_node

# Test the transformer
print("Original:", src_code_2)
transformer = ChainSimplifier()
new_tree = src_tree_2.visit(transformer)
print("Transformed:", new_tree.code)

Original: ds.search(source_id='any').search(source_id='.*').to_dataset_dict()
Found chain: ds.search(source_id='any') -> .search()
Removing .search(source_id='any') from chain, keeping .search()
Found chain: ds.search(source_id='.*') -> .to_dataset_dict()
Removing .search(source_id='.*') from chain, keeping .to_dataset_dict()
Transformed: ds.to_dataset_dict()
